# Ecosystem services widgets  
This contains two widgets with independent data:
1. Above/below ground organic carbon
2. Commercial fishing restoration

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import fiona

## Above/below ground organic carbon data

In [2]:
gdb_file =f'../../../data/Typology_and_Restoration_Potential/Data/MOW_Global_Mangrove_Restoration_20190411.gdb'
all_data = gpd.read_file(gdb_file, driver='FileGDB', layer=0)
all_data.head()

,Country,Tot_Restor,Restor_pct,Rest_Area_Loss,Area_loss_pct,Rest_Area_Dgrd,Area_dgrd_pct,Total_2016,Mean_Score,SOC,AGB,People,Fish_Score,Fish_Score_Inv,Shape_Length,Shape_Area,Loss_Driver,geometry
0,American Samoa,0.00000,0,0.00,0,0.00,0,18.74,0,0.000000,0.000000,0,109000000.0,200000.0,5.350212e+04,1.638363e+07,No Data,"MULTIPOLYGON (((-19011686.866 -1613074.241, -1..."
1,Angola,580.11556,4,671.43,5,68.92,0,13286.05,62,220274.238512,30809.263129,3800,-1.0,-1.0,1.383794e+06,2.785231e+10,NPC,"MULTIPOLYGON (((1550210.491 -1230695.594, 1549..."
2,Anguilla,0.08128,4,1.27,59,0.00,0,0.87,24,407.670000,6.524752,0,1000000.0,2000000.0,2.097797e+04,1.419573e+06,No Data,"MULTIPOLYGON (((-7025249.376 2058315.576, -702..."
3,Antigua and Barbuda,14.15571,2,20.19,2,0.00,0,886.30,22,9031.556968,1078.042000,500,8000000.0,13000000.0,1.888816e+05,1.361710e+09,No Data,"MULTIPOLYGON (((-6873186.951 1921601.093, -688..."
4,Aruba,19.38207,36,20.55,38,0.00,0,33.79,46,9816.581150,1374.874165,0,71000000.0,119000000.0,3.798229e+04,2.166772e+07,No Data,"MULTIPOLYGON (((-7781925.061 1393950.107, -778..."


In [4]:
carbon_data = all_data[['Country', 'SOC', 'AGB']].copy()
carbon_data

,Country,SOC,AGB
0,American Samoa,0.000000e+00,0.000000e+00
1,Angola,2.202742e+05,3.080926e+04
2,Anguilla,4.076700e+02,6.524752e+00
3,Antigua and Barbuda,9.031557e+03,1.078042e+03
4,Aruba,9.816581e+03,1.374874e+03
...,...,...,...
103,Vanuatu,5.545036e+03,8.574929e+02
104,Venezuela,4.366034e+06,9.770106e+05
105,Vietnam,5.100087e+06,1.311131e+06
106,"Virgin Islands, U.S.",1.609178e+03,2.919104e+02


In [6]:
carbon_long = pd.melt(carbon_data, id_vars='Country')
carbon_long.rename(columns={'variable':'indicator'}, inplace=True)
carbon_long

,Country,indicator,value
0,American Samoa,SOC,0.000000e+00
1,Angola,SOC,2.202742e+05
2,Anguilla,SOC,4.076700e+02
3,Antigua and Barbuda,SOC,9.031557e+03
4,Aruba,SOC,9.816581e+03
...,...,...,...
211,Vanuatu,AGB,8.574929e+02
212,Venezuela,AGB,9.770106e+05
213,Vietnam,AGB,1.311131e+06
214,"Virgin Islands, U.S.",AGB,2.919104e+02


In [7]:
# Import locations to get staging ids
locations = pd.read_csv('../../../data/staging_locations.csv')
locations = locations[locations['location_type'] == 'country']
locations = locations[['id', 'name', 'iso']].copy()
locations.head()

,id,name,iso
159,1402,Dominican Republic,DOM
160,1401,Colombia,COL
161,1400,"Congo, DRC",COD
162,1399,Australia,AUS
163,1398,Angola,AGO


In [8]:
# Import gadm file to retrieve ISO codes
gadm = gpd.read_file('../../../data/gadm36_level0_original/gadm36_level0_original.shp')
gadm.head()

,GID_0,NAME_0,AREA_KM2,MOL_ID,Shape_Leng,Shape_Area,geometry
0,ABW,Aruba,1.819384e+02,1,0.963634,0.015131,"POLYGON ((-69.97820 12.46986, -69.97847 12.469..."
1,AFG,Afghanistan,6.438575e+05,2,57.103371,62.749594,"POLYGON ((68.52644 31.75435, 68.53852 31.75457..."
2,AGO,Angola,1.247422e+06,3,73.796528,103.818655,"MULTIPOLYGON (((11.73347 -16.67255, 11.73347 -..."
3,AIA,Anguilla,8.330331e+01,4,1.318321,0.007116,"MULTIPOLYGON (((-63.42375 18.58903, -63.42375 ..."
4,ALA,Åland,1.506261e+03,5,42.232199,0.243769,"MULTIPOLYGON (((21.32195 59.74986, 21.32195 59..."


In [16]:
#Prepare locations data
rp = all_data[['Country']]

isocodes = gadm[['GID_0', 'NAME_0']]

rp = pd.merge(rp, isocodes, how='left', left_on='Country', right_on='NAME_0')
rp.head()


,Country,GID_0,NAME_0
0,American Samoa,ASM,American Samoa
1,Angola,AGO,Angola
2,Anguilla,AIA,Anguilla
3,Antigua and Barbuda,ATG,Antigua and Barbuda
4,Aruba,ABW,Aruba


In [17]:
rp.GID_0[30] = 'TLS' # East Timor is now called Timor-Leste and the iso is TLS


In [18]:

rp = rp.drop(columns='NAME_0').rename(columns={'GID_0':'iso'})
rp.head()

,Country,iso
0,American Samoa,ASM
1,Angola,AGO
2,Anguilla,AIA
3,Antigua and Barbuda,ATG
4,Aruba,ABW


In [21]:
#Add staging locations
final_locations = pd.merge(rp, locations[['iso', 'id']], how='left', on='iso')
final_locations

,Country,iso,id
0,American Samoa,ASM,NaN
1,Angola,AGO,1398.0
2,Anguilla,AIA,NaN
3,Antigua and Barbuda,ATG,1370.0
4,Aruba,ABW,NaN
...,...,...,...
103,Vanuatu,VUT,1394.0
104,Venezuela,VEN,1362.0
105,Vietnam,VNM,1364.0
106,"Virgin Islands, U.S.",VIR,1397.0


In [25]:
df_final = pd.merge(final_locations, carbon_long, on='Country', how='right')
df_final.rename(columns={'id':'location_id'}, inplace=True)
df_final

,Country,iso,location_id,indicator,value
0,American Samoa,ASM,NaN,SOC,0.000000e+00
1,Angola,AGO,1398.0,SOC,2.202742e+05
2,Anguilla,AIA,NaN,SOC,4.076700e+02
3,Antigua and Barbuda,ATG,1370.0,SOC,9.031557e+03
4,Aruba,ABW,NaN,SOC,9.816581e+03
...,...,...,...,...,...
211,Vanuatu,VUT,1394.0,AGB,8.574929e+02
212,Venezuela,VEN,1362.0,AGB,9.770106e+05
213,Vietnam,VNM,1364.0,AGB,1.311131e+06
214,"Virgin Islands, U.S.",VIR,1397.0,AGB,2.919104e+02


In [28]:
df_final[~df_final['location_id'].isna()][['location_id', 'indicator', 'value']].to_csv('../../../data/ecosystem_services_carbon_data.csv', index=False)